In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from datetime import date
import json
from pprint import pprint
# Import API key
from attom_key import api_key



In [2]:
atx_zip_codes = [78731]



headers = {"apikey": api_key, 
           "accept": "application/json"}

params_zip = {"propertytype": "SFR",
              "page": "1",
              
              "pagesize": "500",
             }

zip_id_url = "https://search.onboard-apis.com/propertyapi/v1.0.0/property/address"

propid_list = []  

for zipcodes in atx_zip_codes:
    params_zip["postalcode"] = zipcodes
    
    zip_results = requests.get(zip_id_url, headers=headers, params=params_zip).json()
    
    results = zip_results["property"]

    for i in range(len(results)):
        propid_zip = {"propid": results[i]["identifier"]["obPropId"], "zip": results[i]["address"]["postal1"]}
        propid_list.append(propid_zip)



In [3]:
atx_zips_df = pd.DataFrame(propid_list)
                                                      
atx_zips_df.count()

propid    500
zip       500
dtype: int64

In [4]:
atx_zips_df["appraisal_2017"] = ""
atx_zips_df["appraisal_2016"] = ""
atx_zips_df["appraisal_2015"] = ""
atx_zips_df["appraisal_2014"] = ""
atx_zips_df["appraisal_2013"] = ""

atx_zips_df.head()

,propid,zip,appraisal_2017,appraisal_2016,appraisal_2015,appraisal_2014,appraisal_2013
0,16630028148453,78731,,,,,
1,482595048453,78731,,,,,
2,16630028348453,78731,,,,,
3,16630027348453,78731,,,,,
4,16630027448453,78731,,,,,


In [5]:
headers = {"apikey": api_key, 
           "accept": "application/json"}

params_ass = {}

ass_url = "https://search.onboard-apis.com/propertyapi/v1.0.0/assessmenthistory/detail"

# use iterrows to iterate through pandas dataframe
for index, row in atx_zips_df.iterrows():

    # get restaurant type from df
    propid = row["propid"]

    # add keyword to params dict
    params_ass["id"] = propid

    # assemble url and make API request
    ass_results = requests.get(ass_url, headers=headers, params=params_ass).json()
    
    # extract results
    results_ass = ass_results["property"][0]["assessmenthistory"]
    

    
    try:

        atx_zips_df.loc[index,"appraisal_2016"] = results_ass[0]["appraised"]["apprttlvalue"]
        
        atx_zips_df.loc[index,"appraisal_2017"] = results_ass[1]["appraised"]["apprttlvalue"]
        
        atx_zips_df.loc[index,"appraisal_2015"] = results_ass[2]["appraised"]["apprttlvalue"]
      
        atx_zips_df.loc[index,"appraisal_2014"] = results_ass[3]["appraised"]["apprttlvalue"]
      
        atx_zips_df.loc[index,"appraisal_2013"] = results_ass[4]["appraised"]["apprttlvalue"]

    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        


Missing field/result... skipping.
Missing field/result... skipping.


In [6]:
atx_zips_df.count()

propid            500
zip               500
appraisal_2017    500
appraisal_2016    500
appraisal_2015    500
appraisal_2014    500
appraisal_2013    500
dtype: int64

In [7]:
ohtwo_df = atx_zips_df
ohtwo_df.count()

propid            500
zip               500
appraisal_2017    500
appraisal_2016    500
appraisal_2015    500
appraisal_2014    500
appraisal_2013    500
dtype: int64

In [8]:
ohtwo_df.replace("", np.nan, inplace=True)
ohtwo_df.dropna(axis=0, inplace=True)
ohtwo_df.count()

propid            498
zip               498
appraisal_2017    498
appraisal_2016    498
appraisal_2015    498
appraisal_2014    498
appraisal_2013    498
dtype: int64

In [9]:
# atx_zips_df.replace("", np.nan, inplace=True)
# atx_zips_df.dropna(axis=0, inplace=True)
# print(atx_zips_df.count())


In [10]:
ohtwo_df.to_csv("assessment_history_by_zipcode_78731.csv")